In [1]:
# !pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
from fastbook import *
from fastai.vision.widgets import *

In [3]:
# !pip install voila
# !jupyter serverextension enable --sys-prefix voila

In [4]:
# Path
path = Path()

In [5]:
# Carrega Learner
learn_inf = load_learner(path/'export.pkl')

In [6]:
# Widgets
btn_upload = widgets.FileUpload()  # Botão de upload
out_pl = widgets.Output()  # Imagem
lbl_pred = widgets.Label()  # Texto
btn_run = widgets.Button(description='Classificar')  # Botão de classificação

In [7]:
# Define o que o botão de classificação faz
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [8]:
# Exibição
VBox([widgets.Label('Selecione seu urso!'), 
      btn_upload, btn_run, out_pl, lbl_pred])